In [1]:
from nlp_fast_unlearning.utils import prepare_dbpedia, ensure_deterministic, DEVICE, BATCH_SIZE


train_dataloader, val_dataloader, test_dataloader, vocab_size = prepare_dbpedia(for_baseline_only=True)

In [2]:
from nlp_fast_unlearning.baseline_model import TextClassificationModel


model = TextClassificationModel(vocab_size).to(DEVICE)

In [3]:
import time

def train(dataloader, model=model, neggrad=False, log_interval = 1000):
    model.train()
    total_acc, total_count = 0, 0
    start_time = time.time()

    total_loss=0
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        if neggrad is True:
            loss = -criterion(predicted_label, label)
        else:
            loss = criterion(predicted_label, label)
        loss.backward()
        if neggrad is False:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        total_loss+=loss.item()
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| loss {:8.6f}".format(
                    epoch, idx, len(dataloader), total_loss / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader, model=model):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [4]:
import torch

EPOCHS = 10
LR = 5

criterion = model.criterion
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

In [5]:
ensure_deterministic()


for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(val_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch   1 |  1000/ 2079 batches | loss 0.002248
| epoch   1 |  2000/ 2079 batches | loss 0.002902
-----------------------------------------------------------
| end of epoch   1 | time: 31.31s | valid accuracy    0.964 
-----------------------------------------------------------
| epoch   2 |  1000/ 2079 batches | loss 0.000468
| epoch   2 |  2000/ 2079 batches | loss 0.000872
-----------------------------------------------------------
| end of epoch   2 | time: 30.53s | valid accuracy    0.971 
-----------------------------------------------------------
| epoch   3 |  1000/ 2079 batches | loss 0.000350
| epoch   3 |  2000/ 2079 batches | loss 0.000674
-----------------------------------------------------------
| end of epoch   3 | time: 30.55s | valid accuracy    0.976 
-----------------------------------------------------------
| epoch   4 |  1000/ 2079 batches | loss 0.000289
| epoch   4 |  2000/ 2079 batches | loss 0.000578
---------------------------------------------------------

In [6]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.5f}".format(accu_test*100))

Checking the results of test dataset.
test accuracy 97.93429


In [7]:
baseline_name = "DBpedia_baseline.pt"
torch.save(model.state_dict(), baseline_name)